In [8]:
from utils import *
from model import *
import sys
import torch.optim as optim
from torch import nn
import torch
from sklearn.metrics import accuracy_score, classification_report

In [2]:
train_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-train.csv'
test_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-test.csv'
val_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-val.csv'
w2v_file = '../data/ubercorpus.cased.tokenized.word2vec.300d'

In [17]:
class Config(object):
    embed_size = 300
    hidden_layers = 1
    hidden_size = 64
    output_size = 2
    max_epochs = 1
    hidden_size_linear = 64
    lr = 0.5
    batch_size = 32
    seq_len = None # Sequence length for RNN
    dropout_keep = 0.8


In [4]:
config = Config()
dataset = Dataset(config)
dataset.load_data(w2v_file, train_file, test_file, val_file)

Loaded 281363 training examples
Loaded 119849 test examples
Loaded 119498 validation examples


#### Create Model with specified optimizer and loss function

In [5]:
model = RCNN(config, len(dataset.vocab), dataset.word_embeddings)
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
train_losses = []
val_accuracies = []

In [9]:
def train():
    for i in range(config.max_epochs):
        print ("Epoch: {}".format(i))
        train_loss,val_accuracy = model.run_epoch(dataset.train_iterator, dataset.val_iterator, i)
        train_losses.append(train_loss)
        val_accuracies.append(val_accuracy)

In [10]:
%time train()


Epoch: 0
Reducing LR
Iter: 1
	Average training loss: -0.34465


KeyboardInterrupt: 

In [11]:
def calc_true_and_pred(model, iterator):
    all_preds = []
    all_y = []
    for idx,batch in enumerate(iterator):
        if torch.cuda.is_available():
            x = batch.text.cuda()
        else:
            x = batch.text
        y_pred = model(x)
        predicted = torch.max(y_pred.cpu().data, 1)[1] + 1
        all_preds.extend(predicted.numpy())
        all_y.extend(batch.label.numpy())
    return all_y, all_preds

In [12]:
all_y, all_preds = calc_true_and_pred(model, dataset.test_iterator)

### accuracy score

In [13]:
test_acc = accuracy_score(all_y, np.array(all_preds).flatten())
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

Final Test Accuracy: 0.4834


### confusion matrix

In [16]:
print(classification_report(all_y, np.array(all_preds).flatten()))

              precision    recall  f1-score   support

           1       0.50      0.46      0.48     56013
           2       0.56      0.51      0.53     63836
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

   micro avg       0.48      0.48      0.48    119849
   macro avg       0.26      0.24      0.25    119849
weighted avg       0.53      0.48      0.51    119849



In [7]:
train_acc = evaluate_model(model, dataset.train_iterator)
val_acc = evaluate_model(model, dataset.val_iterator)
test_acc = evaluate_model(model, dataset.test_iterator)

print ('Final Training Accuracy: {:.4f}'.format(train_acc))
print ('Final Validation Accuracy: {:.4f}'.format(val_acc))
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

Final Training Accuracy: 0.4537
Final Validation Accuracy: 0.4525
Final Test Accuracy: 0.4456
